# Run the MNE BIDS Pipeline

In [1]:
import mne_bids_pipeline
from typing import Optional
from mne_bids import BIDSPath
def get_input_fnames_apply_ica(
    *,
    cfg,
    subject: str,
    session: Optional[str],
) -> dict:
    bids_basename = BIDSPath(
        subject=subject,
        session=session,
        task=cfg.task,
        acquisition=cfg.acq,
        recording=cfg.rec,
        space=cfg.space,
        datatype='eeg',
        root=cfg.deriv_root,
        check=False,
    )
    paths = dict()
    paths["ica"] = bids_basename.copy().update(suffix="ica", extension=".fif")
    paths["raw"] = bids_basename.copy().update(suffix="proc-filt_raw", extension=".fif")
    paths["components"] = bids_basename.copy().update(
        processing="ica", suffix="components", extension=".tsv"
    )
    return paths

In [2]:
from mne_bids_pipeline._config_import import _import_config as getConfig

In [3]:
config_path = "./mne-bids/config/mne-bids-pipeline_tests.py"

In [4]:
# set configuration data
cfg = getConfig(
    config_path=config_path
)

In [5]:
# checkup: print config contents
print(cfg)

namespace(study_name='EEG-course-project', bids_root=PosixPath('/home/fspletti/vorlesungen/2023S_Signal_processing_and_Analysis_of_human_brain_potentials_(EEG)/git/eeg_course_project/data/ds003702'), deriv_root=PosixPath('/home/fspletti/vorlesungen/2023S_Signal_processing_and_Analysis_of_human_brain_potentials_(EEG)/git/eeg_course_project/data/ds003702/derivatives/mne-bids-pipeline'), subjects_dir='./data/ds003702/', sessions='all', task='SocialMemoryCuing', runs='all', exclude_runs=None, crop_runs=None, acq=None, proc=None, rec=None, space=None, plot_psd_for_runs='all', subjects=['01'], exclude_subjects=[], process_empty_room=True, process_rest=True, ch_types=['eeg'], data_type='eeg', eog_channels=None, eeg_bipolar_channels=None, eeg_reference='average', eeg_template_montage='standard_1005', drop_channels=['EOG', 'BIP1', 'BIP2', 'BIP3', 'BIP4', 'BIP5', 'BIP6', 'BIP7', 'BIP8', 'BIP9', 'BIP10', 'BIP11', 'BIP12', 'BIP13', 'BIP14', 'BIP15', 'BIP16', 'BIP17', 'BIP18', 'BIP19', 'BIP20', 'BI

In [6]:
# checkup: # print directories of the stored bids root path
cfg.bids_root.parts[-2:]

('data', 'ds003702')

In [7]:
curr_steps = "init"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│20:50:43│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline_tests.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│20:50:43│ ⏳️ Initializing output directories.
└────────┴ done (2s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│20:50:44│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│20:50:44│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│20:50:44│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)


In [8]:
# checkup: check, whether output directory was created
#          The derivatives directory should exist and contain a subfolder per selected subject.
#          The subjects subfolders should be empty if no data were alread generated.
curr_bids_root = cfg.bids_root.__str__()
!ls "{curr_bids_root}"/derivatives/mne-bids-pipeline/

dataset_description.json  sub-01


In [9]:
curr_steps = "preprocessing/_01_data_quality"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│20:52:58│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline_tests.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│20:52:58│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│20:52:59│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ preprocessing/_01_data_quality ──────────────────────────────────────
│20:53:03│ ⏳️ sub-01 Initializing report HDF5 file
│20:53:03│ ⏳️ sub-01 Adding original raw data to report
│20:53:33│ ⏳️ sub-01 Adding config and sys info to report
│20:53:40│ ⏳️ sub-01 Saving report: ]8;id=480688;file:///home/fspletti/vorlesungen/2023S_Signal_processing_and_Analysis_of_human_brain_potentials_(EEG)/git/eeg_course_project/data/ds003702/derivatives/mne-bids-pipeline/sub-01/eeg/sub-01_task-SocialMemoryCuing_report.

In [30]:
# checkup: print some content which was written to file
import h5py
curr_checkup_h5content = h5py.File(f"{curr_bids_root}/derivatives/mne-bids-pipeline/sub-01/eeg/sub-01_task-SocialMemoryCuing_report.h5", 'r')
curr_html_length = len(curr_checkup_h5content["mnepython/key__content/idx_0/key_html"])
htmlContent = ''.join([ chr(curr_checkup_h5content["mnepython/key__content/idx_0/key_html"][idx] ) for idx in range(curr_html_length) ])
print(f"[...]\n{htmlContent[2500:2850]}\n[...]")

print("\n===\nmore data: see at..")
# for more data, open the written html report file
!ls ./"{cfg.bids_root.parts[-2]}"/"{cfg.bids_root.parts[-1]}"/derivatives/mne-bids-pipeline/sub-01/eeg/sub-01_task-SocialMemoryCuing_report.html

[...]
</details>
        <details open>
            <summary><strong>Data</strong></summary>
            <table class="table table-hover table-striped table-sm table-responsive small">
                
                <tr>
                    <th>Sampling frequency</th>
                    <td>500.00 Hz</td>
                </tr>
                
       
[...]

===
more data: see at..
./data/ds003702/derivatives/mne-bids-pipeline/sub-01/eeg/sub-01_task-SocialMemoryCuing_report.html


In [26]:
curr_steps = "preprocessing/_02_head_pos"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│20:04:38│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│20:04:39│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│20:04:39│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ preprocessing/_02_head_pos ──────────────────────────────────────────
│20:04:39│ ⏩ Skipping …
└────────┴ done (1s)


In [ ]:
# checkup: not needed, since this step was skipped

In [27]:
curr_steps = "preprocessing/_03_maxfilter"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│20:05:01│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│20:05:01│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│20:05:02│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ preprocessing/_03_maxfilter ─────────────────────────────────────────
│20:05:02│ ⏩ Skipping …
└────────┴ done (1s)


In [ ]:
# checkup: not needed, since this step was skipped

In [28]:
curr_steps = "preprocessing/_04_frequency_filter"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│20:05:36│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│20:05:36│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│20:05:37│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ preprocessing/_04_frequency_filter ──────────────────────────────────
│20:05:40│ ✅ sub-03 Computation unnecessary (cached) …
│20:05:40│ ✅ sub-01 Computation unnecessary (cached) …
│20:05:40│ ✅ sub-07 Computation unnecessary (cached) …
│20:05:40│ ✅ sub-09 Computation unnecessary (cached) …
│20:05:40│ ✅ sub-10 Computation unnecessary (cached) …
│20:05:40│ ✅ sub-12 Computation unnecessary (cached) …
│20:05:40│ ✅ sub-11 Computation unnecessary (cached) …
│20:05:40│ ✅ sub-13 Computation unnecessary (cached) …

In [ ]:
# checkup: TODO (print spectrum of original and filtered signals, print sample rate of new raw data file)

In [30]:
curr_steps = "preprocessing/_05_make_epochs"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│20:05:57│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│20:05:58│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│20:05:59│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ preprocessing/_05_make_epochs ───────────────────────────────────────
│20:06:03│ ✅ sub-02 Computation unnecessary (cached) …
│20:06:03│ ✅ sub-03 Computation unnecessary (cached) …
│20:06:03│ ✅ sub-04 Computation unnecessary (cached) …
│20:06:03│ ✅ sub-07 Computation unnecessary (cached) …
│20:06:03│ ✅ sub-09 Computation unnecessary (cached) …
│20:06:03│ ✅ sub-11 Computation unnecessary (cached) …
│20:06:03│ ✅ sub-10 Computation unnecessary (cached) …
│20:06:03│ ✅ sub-12 Computation unnecessary (cached) …

In [32]:
curr_steps = "preprocessing/_06a_run_ica"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│20:06:25│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│20:06:25│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│20:06:26│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ preprocessing/_06a_run_ica ──────────────────────────────────────────
│20:06:28│ ✅ sub-04 Computation unnecessary (cached) …
│20:06:28│ ✅ sub-01 Computation unnecessary (cached) …
│20:06:28│ ✅ sub-05 Computation unnecessary (cached) …
│20:06:28│ ✅ sub-07 Computation unnecessary (cached) …
│20:06:28│ ✅ sub-03 Computation unnecessary (cached) …
│20:06:28│ ✅ sub-09 Computation unnecessary (cached) …
│20:06:28│ ✅ sub-10 Computation unnecessary (cached) …
│20:06:28│ ✅ sub-02 Computation unnecessary (cached) …

In [35]:
import mne
import mne_icalabel
from mne.preprocessing import read_ica
import pandas as pd
from mne_bids_pipeline._config_utils import (
    get_subjects,
    get_sessions
)

for subject in get_subjects(cfg):
    for session in get_sessions(cfg):
        paths = get_input_fnames_apply_ica(cfg=cfg, subject=subject, session=session)
        ica = read_ica(paths["ica"])
        raw = mne.io.read_raw_fif(paths["raw"])
        print(str(ica))
        label_results = mne_icalabel.label_components(raw, ica, method="iclabel")
        print(label_results["y_pred_proba"])
        print(label_results["labels"])
        labels = label_results["labels"]
        exclude_idx = [
            idx for idx, label in enumerate(labels) if label not in ["brain", "other"]
        ]
        tsv_data = pd.read_csv(paths["components"], sep="\t")
        print(str(tsv_data))
        tsv_data.loc[exclude_idx, "status"] = "bad"
        print(tsv_data)
        
        tsv_data.to_csv(paths["components"], sep="\t", index=False)
    


<ICA | epochs decomposition, method: picard (fit in 39 iterations on 46632 samples), 8 ICA components (63 PCA components available), channel types: eeg, no sources marked for exclusion>
[0.48660833 0.9977477  0.82226956 0.9747932  0.4546755  0.54981893
 0.62330765 0.9569508 ]
['eye blink', 'eye blink', 'brain', 'brain', 'brain', 'eye blink', 'brain', 'brain']
   component type            description status  status_description
0          0  ica  Independent Component   good                 NaN
1          1  ica  Independent Component   good                 NaN
2          2  ica  Independent Component   good                 NaN
3          3  ica  Independent Component   good                 NaN
4          4  ica  Independent Component   good                 NaN
5          5  ica  Independent Component   good                 NaN
6          6  ica  Independent Component   good                 NaN
7          7  ica  Independent Component   good                 NaN
   component type         

In [ ]:
curr_steps = "preprocessing/_07a_apply_ica"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

┌────────┬ Welcome aboard MNE-BIDS-Pipeline! 👋 ────────────────────────────────
│20:11:19│ 📝 Using configuration: ./mne-bids/config/mne-bids-pipeline.py
└────────┴ 
┌────────┬ init/_01_init_derivatives_dir ───────────────────────────────────────
│20:11:19│ ✅ Output directories already exist …
└────────┴ done (1s)
┌────────┬ init/_02_find_empty_room ────────────────────────────────────────────
│20:11:20│ ⏩ Skipping, empty-room data only relevant for MEG …
└────────┴ done (1s)
┌────────┬ preprocessing/_07a_apply_ica ────────────────────────────────────────
│20:11:23│ ⏳️ sub-01 Input: sub-01_task-SocialMemoryCuing_epo.fif
│20:11:23│ ⏳️ sub-01 Output: sub-01_task-SocialMemoryCuing_proc-ica_epo.fif
│20:11:23│ ⏳️ sub-02 Input: sub-02_task-SocialMemoryCuing_epo.fif
│20:11:23│ ⏳️ sub-02 Output: sub-02_task-SocialMemoryCuing_proc-ica_epo.fif
│20:11:23│ ⏳️ sub-03 Input: sub-03_task-SocialMemoryCuing_epo.fif
│20:11:23│ ⏳️ sub-03 Output: sub-03_task-SocialMemoryCuing_proc-ica_epo.fif
│20:11:23│ ⏳️

In [ ]:
curr_steps = "preprocessing/_08_ptp_reject"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

In [ ]:
curr_steps = "sensor"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}

In [ ]:
curr_steps = "source"
!mne_bids_pipeline --config {config_path} --steps {curr_steps}